In [1]:
!date

Fri Oct 23 08:49:38 PDT 2020


# 10x correlation on isoforms

In [2]:
import anndata
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import scanpy as scanp
from scipy.stats import ks_2samp, ttest_ind
from scipy import stats
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE
from umap import UMAP
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from matplotlib import cm
import ast
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('/home/sina/projects/mop/BYVSTZP_2020/trackfig')
from trackfig.utils import get_notebook_name
from trackfig.trackfig import trackfig 

TRACKFIG = "/home/sina/projects/mop/BYVSTZP_2020/trackfig.txt"
NB = get_notebook_name()

def nd(arr):
    return np.asarray(arr).reshape(-1)

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

In [3]:
path = "../../data/10xv3"
file_paths = glob.glob(path + "/tcc_gene_out*/counts_filtered/adata.h5ad")

In [4]:
file_paths = np.sort(file_paths).tolist()

In [5]:
file_paths

['../../data/10xv3/tcc_gene_out_L8TX_181211_01_A01_S01_L003/counts_filtered/adata.h5ad',
 '../../data/10xv3/tcc_gene_out_L8TX_181211_01_B01_S01_L003/counts_filtered/adata.h5ad',
 '../../data/10xv3/tcc_gene_out_L8TX_181211_01_C01_S01_L003/counts_filtered/adata.h5ad',
 '../../data/10xv3/tcc_gene_out_L8TX_181211_01_D01_S01_L003/counts_filtered/adata.h5ad',
 '../../data/10xv3/tcc_gene_out_L8TX_181211_01_E01_S01_L003/counts_filtered/adata.h5ad',
 '../../data/10xv3/tcc_gene_out_L8TX_181211_01_F01_S01_L003/counts_filtered/adata.h5ad',
 '../../data/10xv3/tcc_gene_out_L8TX_181211_01_G12_S01_L003/counts_filtered/adata.h5ad',
 '../../data/10xv3/tcc_gene_out_L8TX_181211_01_H12_S01_L003/counts_filtered/adata.h5ad',
 '../../data/10xv3/tcc_gene_out_L8TX_190430_01_A08_S01_L003/counts_filtered/adata.h5ad',
 '../../data/10xv3/tcc_gene_out_L8TX_190430_01_B08_S01_L003/counts_filtered/adata.h5ad',
 '../../data/10xv3/tcc_gene_out_L8TX_190430_01_F08_S01_L003/counts_filtered/adata.h5ad',
 '../../data/10xv3/tc

In [6]:
batch_id = [
"3L8TX_181211_01_A01",
"4L8TX_181211_01_B01",
"5L8TX_181211_01_C01",
"6L8TX_181211_01_D01",
"7L8TX_181211_01_E01",
"8L8TX_181211_01_F01",
"1L8TX_181211_01_G12",
"2L8TX_181211_01_H12",
"9L8TX_190430_01_A08",    
"10L8TX_190430_01_B08",
"11L8TX_190430_01_F08",    
"12L8TX_190430_01_G08"]

In [7]:
%%time
files = []
for fidx, f in enumerate(file_paths):
    print("{} of {}".format(fidx+1, len(file_paths)))
    
    adata = anndata.read_h5ad(f)
    
    adata.var["transcript_ids"] = adata.var["transcript_ids"].apply(lambda x: ast.literal_eval(x))
    adata.var["ec_len"] = adata.var["transcript_ids"].apply(lambda x: len(x))
    
    adata = adata[:,adata.var.ec_len==1]
    adata.var["transcript_id"] = adata.var.transcript_ids.apply(lambda x: x[0])
    adata.var["transcript_id_nov"] = adata.var["transcript_id"].str.slice(0,18)
    
    
    files.append(adata)

1 of 12


Trying to set attribute `.var` of view, copying.


2 of 12


Trying to set attribute `.var` of view, copying.


3 of 12


...

11 of 12


Trying to set attribute `.var` of view, copying.


12 of 12


Trying to set attribute `.var` of view, copying.


CPU times: user 22min 19s, sys: 1min 1s, total: 23min 21s
Wall time: 23min 57s


In [8]:
%%time
adata = files[0].concatenate(*files[1:], batch_categories=batch_id)

CPU times: user 3.84 s, sys: 1.08 s, total: 4.92 s
Wall time: 4.97 s


In [9]:
adata

AnnData object with n_obs × n_vars = 169200 × 111079
    obs: 'batch'
    var: 'transcript_ids', 'ec_len', 'transcript_id', 'transcript_id_nov'

In [12]:
adata.write("../../data/notebook/revision/10xv3_unique_isoforms.h5ad")

In [ ]:
#adata = anndata.read_h5ad("../../data/notebook/revision/10xv3_unique_isoforms.h5ad")

In [ ]:
cols = ["transcript_id", "gene_id", "gene_name", "transcript_name", "chromosome", "start", "stop", "strand"]
t2g = pd.read_csv("../../reference/t2g.txt", sep="\t", header=None, names=cols)
t2g["transcript_id"] = t2g["transcript_id"].str.slice(0,18)
t2g["gene_id"] = t2g["gene_id"].str.slice(0,18)
t2g["gene_id"] = t2g["gene_name"] + "_" + t2g["gene_id"]

In [ ]:
t2g.index = t2g.transcript_id.values
t2g["transcript_name_short"] = t2g["transcript_name"]
t2g["transcript_name"] = t2g["transcript_name_short"] + "_" + t2g["transcript_id"]

# Map OBS

In [ ]:
ca = pd.read_csv("../../reference/10xv3_cluster_labels/cluster.annotation.csv", index_col=0)
cm = pd.read_csv("../../reference/10xv3_cluster_labels/cluster.membership.csv", index_col=0)

In [ ]:
adata.obs.head()

In [ ]:
adata = adata[adata.obs.index.isin(cm.index)]

In [ ]:
adata

In [ ]:
adata.obs["cluster_id"] = adata.obs.index.map(cm["x"])

In [ ]:
adata.obs["cluster_label"]  = adata.obs["cluster_id"].map(ca["cluster_label"])
adata.obs["subclass_label"] = adata.obs["cluster_id"].map(ca["subclass_label"])
adata.obs["class_label"]    = adata.obs["cluster_id"].map(ca["class_label"])

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
adata.obs["cluster_id"] = le.fit_transform(adata.obs.cluster_label.values)

le = LabelEncoder()
adata.obs["subclass_id"] = le.fit_transform(adata.obs.subclass_label.values)

le = LabelEncoder()
adata.obs["class_id"] = le.fit_transform(adata.obs.class_label.values)

In [ ]:
adata.obs.head()

# Map Var

In [ ]:
t2g.head()

In [ ]:
adata.var.head()

In [ ]:
adata.var["transcript_name"] = adata.var["transcript_id_nov"].map(t2g["transcript_name"])
adata.var.index = adata.var["transcript_name"].values

In [ ]:
adata.var.head()

In [ ]:
#adata.write("../cell_ranger_annotation/10xv3_isoform.h5ad")

# Compare to smartseq

In [4]:
adata = anndata.read_h5ad("../../data/notebook/10xv3_isoform.h5ad")

In [5]:
def change(x):
    if x=="L5 ET": return "L5 PT"
    return x

In [6]:
adata.obs.subclass_label.value_counts()

L5 IT          29721
Low Quality    22728
L6 CT          12807
L2/3 IT        10915
L6 IT           4445
L5/6 NP         3147
Vip             2966
Lamp5           2357
Sst             1869
L6b              554
Oligo            535
Astro            395
Pvalb            368
Sncg             348
doublet          257
Endo             187
L5 ET            161
OPC              145
Macrophage       122
L6 IT Car3        69
VLMC              55
SMC               11
Name: subclass_label, dtype: int64

In [7]:
adata.obs.cluster_label = adata.obs.cluster_label.apply(change).values
adata.obs.subclass_label = adata.obs.subclass_label.apply(change).values

In [8]:
adata.obs.subclass_label.value_counts()

L5 IT          29721
Low Quality    22728
L6 CT          12807
L2/3 IT        10915
L6 IT           4445
L5/6 NP         3147
Vip             2966
Lamp5           2357
Sst             1869
L6b              554
Oligo            535
Astro            395
Pvalb            368
Sncg             348
doublet          257
Endo             187
L5 PT            161
OPC              145
Macrophage       122
L6 IT Car3        69
VLMC              55
SMC               11
Name: subclass_label, dtype: int64

In [3]:
ss = anndata.read_h5ad("../../data/notebook/revision/no_filter_isoform.h5ad")

In [11]:
ss = ss[:, ss.var.sort_index().index]

In [12]:
ss.X

ArrayView([[-0.02131047,  0.        , -0.03038931, ...,  2.01470867,
             2.25286089,  1.22413896],
           [-0.02131047,  0.        , -0.03038931, ...,  0.50629538,
             0.91804969,  0.31529282],
           [-0.02131047,  0.        , -0.03038931, ...,  0.33585474,
             0.22635451,  0.17731334],
           ...,
           [-0.02131047,  0.        , -0.03038931, ..., -6.34526146,
             0.66179349,  1.38375585],
           [-0.02131047,  0.        , -0.03038931, ...,  0.56053482,
            -0.42612873,  1.2261481 ],
           [-0.02131047,  0.        , -0.03038931, ..., -1.71379761,
            -0.98786612, -0.25959408]])

In [10]:
ss.obs.subclass_label.value_counts()

L5 IT          1571
L6 CT           904
Vip             659
L6b             571
Pvalb           543
L2/3 IT         483
Sst             427
L6 IT           395
Lamp5           377
L5/6 NP         208
Sncg             84
SMC              21
L5 PT            12
Low Quality      12
Astro            10
Endo              7
VLMC              6
L6 IT Car3        5
Name: subclass_label, dtype: int64

In [11]:
adata

AnnData object with n_obs × n_vars = 94162 × 111079
    obs: 'batch', 'cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'subclass_id', 'class_id'
    var: 'transcript_ids', 'ec_len', 'transcript_id', 'transcript_id_nov', 'transcript_name'

In [12]:
ss

AnnData object with n_obs × n_vars = 6295 × 111079
    obs: 'cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'cluster_color', 'size', 'cell_id', 'subclass_id', 'class_id', 'cell_counts'
    var: 'transcript_id', 'gene_id', 'gene_name', 'transcript_name', 'length', 'gene_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    layers: 'X', 'log1p', 'norm'

In [13]:
ss = ss[:,ss.var.sort_index().index]
tenx = adata[:,adata.var.sort_index().index]

In [14]:
tenx = tenx[:,tenx.var.index.isin(ss.var.index)]

In [15]:
tenx = tenx[:,tenx.var.sort_index().index]

In [16]:
tenx

View of AnnData object with n_obs × n_vars = 94162 × 111079
    obs: 'batch', 'cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'subclass_id', 'class_id'
    var: 'transcript_ids', 'ec_len', 'transcript_id', 'transcript_id_nov', 'transcript_name'

In [17]:
ss

View of AnnData object with n_obs × n_vars = 6295 × 111079
    obs: 'cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'cluster_color', 'size', 'cell_id', 'subclass_id', 'class_id', 'cell_counts'
    var: 'transcript_id', 'gene_id', 'gene_name', 'transcript_name', 'length', 'gene_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    layers: 'X', 'log1p', 'norm'

In [18]:
ss.var.head()

,transcript_id,gene_id,gene_name,transcript_name,length,gene_counts,highly_variable,means,dispersions,dispersions_norm
0610005C13Rik-201_ENSMUST00000209416,ENSMUST00000209416,ENSMUSG00000109644,0610005C13Rik_ENSMUSG00000109644,0610005C13Rik-201_ENSMUST00000209416,856,10.149507,False,1.611014e-03,1.382855,-0.505609
0610005C13Rik-202_ENSMUST00000209510,ENSMUST00000209510,ENSMUSG00000109644,0610005C13Rik_ENSMUSG00000109644,0610005C13Rik-202_ENSMUST00000209510,671,0.000000,False,1.000000e-12,NaN,NaN
0610005C13Rik-203_ENSMUST00000209910,ENSMUST00000209910,ENSMUSG00000109644,0610005C13Rik_ENSMUSG00000109644,0610005C13Rik-203_ENSMUST00000209910,553,15.954428,False,2.531254e-03,1.198419,-0.606426
0610005C13Rik-204_ENSMUST00000210383,ENSMUST00000210383,ENSMUSG00000109644,0610005C13Rik_ENSMUSG00000109644,0610005C13Rik-204_ENSMUST00000210383,897,1131.564331,False,1.653077e-01,4.382320,1.133968
0610005C13Rik-205_ENSMUST00000210569,ENSMUST00000210569,ENSMUSG00000109644,0610005C13Rik_ENSMUSG00000109644,0610005C13Rik-205_ENSMUST00000210569,963,0.000000,False,1.000000e-12,NaN,NaN


In [19]:
tenx.var.head()

,transcript_ids,ec_len,transcript_id,transcript_id_nov,transcript_name
0610005C13Rik-201_ENSMUST00000209416,['ENSMUST00000209416.1'],1,ENSMUST00000209416.1,ENSMUST00000209416,0610005C13Rik-201_ENSMUST00000209416
0610005C13Rik-202_ENSMUST00000209510,['ENSMUST00000209510.1'],1,ENSMUST00000209510.1,ENSMUST00000209510,0610005C13Rik-202_ENSMUST00000209510
0610005C13Rik-203_ENSMUST00000209910,['ENSMUST00000209910.1'],1,ENSMUST00000209910.1,ENSMUST00000209910,0610005C13Rik-203_ENSMUST00000209910
0610005C13Rik-204_ENSMUST00000210383,['ENSMUST00000210383.1'],1,ENSMUST00000210383.1,ENSMUST00000210383,0610005C13Rik-204_ENSMUST00000210383
0610005C13Rik-205_ENSMUST00000210569,['ENSMUST00000210569.1'],1,ENSMUST00000210569.1,ENSMUST00000210569,0610005C13Rik-205_ENSMUST00000210569


# Process

In [20]:
del adata

In [21]:
from sklearn.preprocessing import normalize

In [22]:
tenx.layers["X"] = tenx.X
tenx.layers["norm"] = normalize(tenx.X, norm='l1', axis=1)*1000000
tenx.layers["log1p"] = csr_matrix(np.log1p(tenx.layers["norm"]))

In [23]:
from sklearn.preprocessing import scale

In [24]:
%%time
mat = tenx.layers["log1p"].todense()
mtx = scale(mat, axis=0, with_mean=True, with_std=True, copy=True)
tenx.X = mtx

CPU times: user 38min 36s, sys: 39min 51s, total: 1h 18min 27s
Wall time: 1h 18min 37s


In [25]:
tenx.write("../../data/notebook/revision/10xv3_isoform_processed.h5ad")

In [25]:
del mat